Copyright (c) 2024 Houssem Ben Braiek, Emilio Rivera-Landos, IVADO, SEMLA

Permission is hereby granted, free of charge, to any person obtaining a copy
of this software and associated documentation files (the "Software"), to deal
in the Software without restriction, including without limitation the rights
to use, copy, modify, merge, publish, distribute, sublicense, and/or sell
copies of the Software, and to permit persons to whom the Software is
furnished to do so, subject to the following conditions:

The above copyright notice and this permission notice shall be included in all
copies or substantial portions of the Software.

THE SOFTWARE IS PROVIDED "AS IS", WITHOUT WARRANTY OF ANY KIND, EXPRESS OR
IMPLIED, INCLUDING BUT NOT LIMITED TO THE WARRANTIES OF MERCHANTABILITY,
FITNESS FOR A PARTICULAR PURPOSE AND NONINFRINGEMENT. IN NO EVENT SHALL THE
AUTHORS OR COPYRIGHT HOLDERS BE LIABLE FOR ANY CLAIM, DAMAGES OR OTHER
LIABILITY, WHETHER IN AN ACTION OF CONTRACT, TORT OR OTHERWISE, ARISING FROM,
OUT OF OR IN CONNECTION WITH THE SOFTWARE OR THE USE OR OTHER DEALINGS IN THE
SOFTWARE.

In [51]:
#LOAD ENV VARIABLES
import os
from dotenv import load_dotenv
load_dotenv()
MLFLOW_URI = os.getenv("MLFLOW_URI")

In [52]:
%load_ext autoreload
%autoreload 2
# TODO: comment out if you want to check info logs
# import logging
# logging.basicConfig(level=logging.INFO)
from sklearn.cluster import KMeans, MiniBatchKMeans
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import RocCurveDisplay
from sklearn.neural_network import MLPClassifier
from sklearn.pipeline import Pipeline
from sklearn.tree import DecisionTreeClassifier
import sys,os
from lightgbm import LGBMRegressor

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [53]:
# Get the current directory path
current_directory = os.getcwd()

# Get the grandparent directory
grandparent_dir = os.path.abspath(os.path.join(current_directory, '..', 'src'))

# Add the parent directory to sys.path
parent_dir = os.path.join(grandparent_dir, 'flight_delay')
sys.path.append(parent_dir)



In [54]:
import pre_processing,train_utils,features

In [55]:
sys.path.append(os.path.join(parent_dir, 'mlflow'))

In [56]:
# # TODO(PARTICIPANT): Update IF NEEDED the data_dir
# dataset, dvc_info = load_dataset_from_dvc(data_dir="../data")

#load the data
data_path = '../data/flights_delay_dataset.csv'
df_flights = pre_processing.load_data(data_path)

In [57]:
df = df_flights[df_flights["covid_data"] == 0]

train_start = "2019-01-01"
train_end   = "2023-01-01"

test_start = "2023-01-01"
test_end   = "2023-09-01"

target_col = "ARR_DELAY"
split_col  = "FL_DATE"

X_train,y_train,X_test,y_test = train_utils.train_test_split_func(df,train_start,train_end,test_start,test_end,split_col,target_col)

In [58]:
numerical_cols = features.numerical_cols
categorical_cols = features.categorical_cols
time_cols = features.time_cols
passthrough_cols = features.passthrough_cols

features_list = numerical_cols + categorical_cols + time_cols + passthrough_cols

predicted_var = "predicted_arrival_delay"

#### Creating info dictionaries

Notice we add information about the DVC file:

In [59]:
import mlflow
from mlflow.models import infer_signature
import mlflow.sklearn
mlflow.set_tracking_uri(MLFLOW_URI)

from tracking import Experiment

## Iteration 1
In this initial iteration, we prepare the dataset for binary classification modeling and proceed to test our first baseline model, the Decision Tree, using our library.

In [60]:
experiment_name = 'lightGBM'
experiment = Experiment(MLFLOW_URI, experiment_name)

In [61]:
X_train,y_train,X_test,y_test = train_utils.train_test_split_func(df,train_start,train_end,test_start,test_end,split_col,target_col)

In [62]:
results = train_utils.train_model_with_tracking(experiment,LGBMRegressor(),LGBMRegressor.__name__, X_train, y_train,X_test,y_test,categorical_cols, numerical_cols, time_cols, passthrough_cols)

/home/ivamlops/flight_delay_prediction/.venv/lib/python3.10/site-packages/sklearn/utils/deprecation.py:151: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(


[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.020773 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 1919
[LightGBM] [Info] Number of data points in the train set: 1409857, number of used features: 774
[LightGBM] [Info] Start training from score 6.068654


/home/ivamlops/flight_delay_prediction/.venv/lib/python3.10/site-packages/sklearn/utils/deprecation.py:151: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(
/home/ivamlops/flight_delay_prediction/.venv/lib/python3.10/site-packages/sklearn/utils/deprecation.py:151: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(
/home/ivamlops/flight_delay_prediction/.venv/lib/python3.10/site-packages/mlflow/types/utils.py:452: UserWarning: Hint: Inferred schema contains integer column(s). Integer columns in Python cannot represent missing values. If your input data contains missing values at inference time, it will be encoded as floats and will cause a schema enforcement error. The best way to avoid this problem is to infer the model schema based on a realistic data sample (training dataset) that includes missing values. Alternatively, you can declare integer 

🏃 View run run_LGBMRegressor_12a80ba0 at: https://mlflow.p02-21.labs.ivamlops.ca/#/experiments/1/runs/fc9dcfd23c9a408795eb22df73fb5381
🧪 View experiment at: https://mlflow.p02-21.labs.ivamlops.ca/#/experiments/1


In [63]:
results

[{'regressor': 'LGBMRegressor',
  'MAE_train': 24.114304280561978,
  'MAE_test': 26.803231543466417}]

## Iteration 2:
The second iteration is about testing multiple models and compare their performance. 

In [ ]:
# experiment_name = 'classic_classif_exp'
# experiment = Experiment(MLFLOW_URI, experiment_name)
# classifiers_list = [DecisionTreeClassifier, MLPClassifier, RandomForestClassifier]
# results = train_and_evaluate_with_tracking(dataset, feature_names, classifiers_list, experiment, classic_ds_info)
# print_accuracy_results(results)

In [ ]:
# explore_best_runs(experiment)

## Iteration 3
The third iteration is about trying out advanced feature engineering with classifier model: Mini-Batch KMeans and Random Forest. 

In [ ]:
# experiment_name = 'random_forest_minibatch_kmeans_tryout'
# experiment = Experiment(MLFLOW_URI, experiment_name)
# experiment_id = mlflow.set_experiment(experiment.name).experiment_id

In [ ]:
# adv_feature_names = AdvFeatureNames(
#     person_info_cols_num, person_info_cols_cat, num_cols_wo_customer, cat_cols_wo_customer
# )

In [ ]:
# with mlflow.start_run(experiment_id=experiment_id):
#     model = Pipeline(
#         [
#             ('input_transformer', make_advanced_data_transformer(adv_feature_names, MiniBatchKMeans())), 
#             ('classifier', RandomForestClassifier()),
#         ]
#     )
#     model.fit(dataset.train_x, dataset.train_y)
#     results = accuracy_evaluation(model, dataset)
#     mlflow.log_metric("train_accuracy", results['train'])
#     mlflow.log_metric("valid_accuracy", results['val'])
#     mlflow.log_metric("test_accuracy", results['test'])
#     print_accuracy_results(results)
#     sample = dataset.train_x.sample(5)
#     signature = infer_signature(sample, 
#                                 model.predict(sample))
#     mlflow.sklearn.log_model(model, "model", 
#                              input_example=sample,
#                              signature=signature,
#                              )
#     mlflow.log_dict(adv_ds_info, "data.yml")
#     display = RocCurveDisplay.from_predictions(dataset.val_y.values,
#                                             model.predict_proba(dataset.val_x)[:,1])
#     mlflow.log_figure(display.figure_, 'plots/ValidRocCurveDisplay.png')
#     display = RocCurveDisplay.from_predictions(dataset.test_y.values,
#                                             model.predict_proba(dataset.test_x)[:,1])
#     mlflow.log_figure(display.figure_, 'plots/TestRocCurveDisplay.png')

## Iteration 4
The fourth iteration is about testing advanced feature engineering with multiple models and comparing their performance. 

In [ ]:
# experiment_name = 'adv_classifier_exp'
# experiment = Experiment(MLFLOW_URI, experiment_name)

In [ ]:
# classifiers_list = [DecisionTreeClassifier, RandomForestClassifier]
# clustering_algos_list = [KMeans, MiniBatchKMeans]
# results = advanced_train_and_evaluate_with_tracking(
#     dataset, adv_feature_names, clustering_algos_list, 
#     classifiers_list, experiment, adv_ds_info
# )
# print_accuracy_results(results)

In [ ]:
# best_run = get_best_run(experiment)

## Final Step
It is important now to register our best advanced model into model registry. We will show how to transition to staging, as well as promoting to production using bank-marketing library, however, these steps should be done after careful validation.

In [ ]:
# import bank_marketing.mlflow.registry as registry

In [ ]:
# model_name = "camp-accept-predictor"

In [ ]:
# model_version = registry.register_model_from_run(experiment.tracking_server_uri, best_run, model_name)

In [ ]:
# registry.transition_model_to_staging(experiment.tracking_server_uri, model_name, model_version.version)

In [ ]:
# model_version = registry.promote_model_to_production(experiment.tracking_server_uri, model_name, f"{model_name}-production")

In [ ]:
# registry.set_active_production_model(experiment.tracking_server_uri,  f"{model_name}-production", model_version.version)